In [1]:
import json 
import spotipy
import pickle
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd

In [2]:
# read data
df = pd.read_csv('../data/df_final2.csv')
df.head()

,Unnamed: 0,name,artist,cluster_main,sub_cluster,speechiness,liveness,valence,tempo,id
0,0,Se Eu Quiser Falar Com Deus,Gilberto Gil,0,3,0.053496,0.987706,0.296665,0.456665,1n7JnwviZ7zf0LR1tcGFq7
1,1,Mulher Eu Sei,Chico César,0,3,0.027362,0.384284,0.522478,0.432506,3mXqOdlLE1k67WsAxryPFs
2,2,Rosa Morena,Kurt Elling,0,0,0.017199,0.095380,0.396105,0.528079,7bSzjzjTkWT2CkIPPdp0eA
3,3,Desafinado,Stan Getz,0,0,0.047241,0.098453,0.417858,0.676409,5pstSsK0G4q86gdyy1Spqh
4,4,Madalena,Maria Gasolina,0,2,0.035627,0.103575,0.688212,0.342801,33u3dO0JWjerYpFN61J3ug


In [8]:
df.cluster_main.unique()

array([0, 1, 3, 2, 4])

In [9]:
df.sub_cluster.unique()

array([3, 0, 2, 1])

In [3]:
# specify path to spotify app login json with all correct info
login_file = '../../api_spotify.json'

scope = 'playlist-modify-public'
redirectUri = 'http://localhost:8080/' # as specified in the app settings

with open(login_file) as f:
    login = json.load(f)
    username = login['username']
    client_id = login['client_id']
    client_secret = login['client_secret']

token = SpotifyOAuth(scope=scope,
                     username=username,
                     client_id=client_id,
                     client_secret=client_secret,
                     redirect_uri=redirectUri,
                     )

spotifyObject = spotipy.Spotify(auth_manager = token)

In [4]:
def create_playlist():
# When this code is run, either a browser window will open, or in Google Colab a URL will appear
# This URL will ask you to sign in to Spotify
# After signing in, copy the URL you are redirected to and paste it in the box that pops up
    
    id_list = list(cl.sample(samplesize).id)
    
    # Spotify needs cetain text wrappers around ID numbers to identify the target as a track, a playlist, etc.
    uris = [f'spotify:track:{str(id).strip()}' for id in id_list]
    playlist_collection[playlist_name] = playlist_id

    try:
        spotifyObject.user_playlist_add_tracks(user=username,
                                           playlist_id=playlist_id,
                                           tracks=uris)
    except:
        pass


def update_playlist():
    with open("./playlists.pkl", "rb") as f:
        playlist_collection = pickle.load(f)
        
    uris = [f'spotify:track:{str(id).strip()}' for id in cl.sample(samplesize).id]
    playlist_id = playlist_collection[playlist_name] # We now replace the 5 songs in playlist 0 with 10 different songs
    
    spotifyObject.user_playlist_replace_tracks(user=username,
                                               playlist_id=playlist_id,
                                               tracks=uris)


def save_playlist_in_file():
    # Playlist IDs can be saved for later
    with open("./playlists.pkl", "wb") as f:
        pickle.dump(playlist_collection, f)


In [7]:
playlist_collection = {}

for main in df.cluster_main.unique():
    for sub in df.sub_cluster.unique():
        print(main, sub)
        cl = df.loc[(df.cluster_main == main) & (df.sub_cluster == sub)]
        playlist_name = f'playlist_{main}_{sub}'
        playlist_description= 'this is a test list'

        if cl.shape[0] > 100:
            samplesize = 100 # Here, we simply take songs randomly from each cluster
        else:
            samplesize = cl.shape[0] # in case the cluster has <100 rows

        playlist_id = spotifyObject.user_playlist_create(user=username,
                                                        name=playlist_name,
                                                        public=True,
                                                        description=playlist_description)['id']
        
        #create_playlist() # use only if playlists are not created yet
        update_playlist() # use if playlists already exist

save_playlist_in_file()

0 3
0 0
0 2
0 1
1 3
1 0
1 2
1 1
3 3
3 0
3 2
3 1
2 3
2 0
2 2
2 1
4 3
4 0
4 2
4 1
